In [2]:
import numpy as np
from cardiacmap.data import CascadeDataFile
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt
import cv2


In [7]:
CascadeDataFile = CascadeDataFile.load_data("2011-08-23_Exp000_Rec112_Cam1-Blue.dat")
signal = CascadeDataFile.signals[0]

In [4]:
def normalize(data):

    return ((data - np.amin(data)) / (np.amax(data) - np.amin(data)) * 256).astype(np.uint8)

    
def normalize_flat(data):

    return ((data - np.amin(data, axis=0)) / (np.amax(data, axis=0) - np.amin(data, axis=0)) * 256).astype(np.uint8)

In [9]:
signal.normalize()


[[3574 3762 3905 ... 2275 2268 2270]
 [3604 3711 3760 ... 2262 2262 2260]
 [3500 3605 3635 ... 2258 2257 2250]
 ...
 [2392 2386 2393 ... 2259 2259 2256]
 [2369 2370 2378 ... 2250 2253 2248]
 [2155 2160 2159 ... 2157 2154 2156]]


In [15]:
signal.perform_average(type="time", sig=4, rad=3)
signal.perform_average(type="spatial", sig=8, rad=6)

(4657, 128, 128)


In [19]:
# Config 1
data = normalize(signal.base_data)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter('video_1.mp4', fourcc, 230, (128, 128), True)

for frame in data:
    writer.write(cv2.applyColorMap(frame, cv2.COLORMAP_RAINBOW))

writer.release()

In [20]:
# Config 2
data = normalize_flat(signal.base_data)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter('video_2.mp4', fourcc, 230, (128, 128), True)

for frame in data:
    writer.write(cv2.applyColorMap(frame, cv2.COLORMAP_RAINBOW))

writer.release()

In [21]:
# Config 3
data = normalize(signal.transformed_data)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter('video_3.mp4', fourcc, 230, (128, 128), True)

for frame in data:
    writer.write(cv2.applyColorMap(frame, cv2.COLORMAP_RAINBOW))

writer.release()

In [22]:
# Config 4
data = normalize_flat(signal.transformed_data)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter('video_4.mp4', fourcc, 230, (128, 128), True)

for frame in data:
    writer.write(cv2.applyColorMap(frame, cv2.COLORMAP_RAINBOW))

writer.release()